import library

In [77]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
from sklearn.preprocessing import KBinsDiscretizer

Load data

In [78]:
train_data = pd.read_csv('./DataMining/input/train.csv')
test_data = pd.read_csv('./DataMining/input/test.csv')
data = pd.concat([train_data, test_data])

data.head()
data.info()
print(len(data))

<class 'pandas.core.frame.DataFrame'>
Index: 129880 entries, 0 to 25975
Data columns (total 25 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Unnamed: 0                         129880 non-null  int64  
 1   id                                 129880 non-null  int64  
 2   Gender                             129880 non-null  object 
 3   Customer Type                      129880 non-null  object 
 4   Age                                129880 non-null  int64  
 5   Type of Travel                     129880 non-null  object 
 6   Class                              129880 non-null  object 
 7   Flight Distance                    129880 non-null  int64  
 8   Inflight wifi service              129880 non-null  int64  
 9   Departure/Arrival time convenient  129880 non-null  int64  
 10  Ease of Online booking             129880 non-null  int64  
 11  Gate location                      129880 non

In [79]:
# data statistics
data.describe()

,Unnamed: 0,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
count,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129880.000000,129487.000000
mean,44158.700000,64940.500000,39.427957,1190.316392,2.728696,3.057599,2.756876,2.976925,3.204774,3.252633,3.441361,3.358077,3.383023,3.350878,3.632114,3.306267,3.642193,3.286326,14.713713,15.091129
std,31207.377062,37493.270818,15.119360,997.452477,1.329340,1.526741,1.401740,1.278520,1.329933,1.350719,1.319289,1.334049,1.287099,1.316252,1.180025,1.266185,1.176669,1.313682,38.071126,38.465650
min,0.000000,1.000000,7.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,16234.750000,32470.750000,27.000000,414.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000
50%,38963.500000,64940.500000,40.000000,844.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,0.000000,0.000000
75%,71433.250000,97410.250000,51.000000,1744.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.000000,12.000000,13.000000
max,103903.000000,129880.000000,85.000000,4983.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,1592.000000,1584.000000


In [80]:
data[['Departure Delay in Minutes', 'Arrival Delay in Minutes']].describe()

,Departure Delay in Minutes,Arrival Delay in Minutes
count,129880.000000,129487.000000
mean,14.713713,15.091129
std,38.071126,38.465650
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,12.000000,13.000000
max,1592.000000,1584.000000


In [81]:
# data statistics non-numeric
data.describe(include=['O'])

,Gender,Customer Type,Type of Travel,Class,satisfaction
count,129880,129880,129880,129880,129880
unique,2,2,2,3,2
top,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied
freq,65899,106100,89693,62160,73452


Tiền xử lý dữ liệu:

In [82]:
# loại bỏ các cột không cần thiết
data=data.drop(['Unnamed: 0','id'],axis=1)

In [83]:
# xử lý dữ liệu thiếu
data.isnull().sum()

Gender                                 0
Customer Type                          0
Age                                    0
Type of Travel                         0
Class                                  0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             393
satisfaction                           0
dtype: int64

In [84]:
# xử lý dữ liệu thiếu cho cột 'Arrival Delay in Minutes'
data['Arrival Delay in Minutes'].fillna(data['Arrival Delay in Minutes'].mean(), inplace=True)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_22016\470683859.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Arrival Delay in Minutes'].fillna(data['Arrival Delay in Minutes'].mean(), inplace=True)


In [85]:
# xử lý duplicate
data.duplicated().sum()

np.int64(0)

In [86]:
# giảm độ lệch của dữ liệu Departure Delay in Minutes, Arrival Delay in Minutes
data['Departure Delay in Minutes'] = data['Departure Delay in Minutes'].apply(lambda x: np.log(x+1))
data['Arrival Delay in Minutes'] = data['Arrival Delay in Minutes'].apply(lambda x: np.log(x+1))

In [87]:
data[['Departure Delay in Minutes', 'Arrival Delay in Minutes']].describe()

,Departure Delay in Minutes,Arrival Delay in Minutes
count,129880.000000,129880.000000
mean,1.233811,1.267978
std,1.617675,1.629508
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,2.564949,2.639057
max,7.373374,7.368340


In [88]:
numeric_columns = ['Age', 'Flight Distance', 'Inflight wifi service', 'Departure/Arrival time convenient']

In [89]:
# xử lý rời rạc hóa
discretizer = KBinsDiscretizer(n_bins=9, encode='ordinal', strategy='uniform')

# Fit and transform the numeric columns
data[numeric_columns] = discretizer.fit_transform(data[numeric_columns])

# Display the first 5 rows to check the discretized data
data.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,0.0,Personal Travel,Eco Plus,0.0,5.0,7.0,3,1,...,5,4,3,4,4,5,5,3.258097,2.944439,neutral or dissatisfied
1,Male,disloyal Customer,2.0,Business travel,Business,0.0,5.0,3.0,3,3,...,1,1,5,3,1,4,1,0.693147,1.945910,neutral or dissatisfied
2,Female,Loyal Customer,2.0,Business travel,Business,2.0,3.0,3.0,2,2,...,5,4,3,4,4,4,5,0.000000,0.000000,satisfied
3,Female,Loyal Customer,2.0,Business travel,Business,0.0,3.0,8.0,5,5,...,2,2,5,3,1,4,2,2.484907,2.302585,neutral or dissatisfied
4,Male,Loyal Customer,6.0,Business travel,Business,0.0,5.0,5.0,3,3,...,3,3,4,4,3,3,3,0.000000,0.000000,satisfied


In [90]:
# xử lý encode dữ liệu
label_encoder = LabelEncoder()
header={
    'Gender': ['Male', 'Female'],
    'Customer Type': ['Loyal Customer', 'disloyal Customer'],
    'Type of Travel': ['Personal Travel', 'Business travel'],
    'Class': ['Eco', 'Eco Plus', 'Business'],
    'satisfaction': ['neutral or dissatisfied', 'satisfied']
}
for key in header:
    data[key] = label_encoder.fit_transform(data[key])
data.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1,0,0.0,1,2,0.0,5.0,7.0,3,1,...,5,4,3,4,4,5,5,3.258097,2.944439,0
1,1,1,2.0,0,0,0.0,5.0,3.0,3,3,...,1,1,5,3,1,4,1,0.693147,1.945910,0
2,0,0,2.0,0,0,2.0,3.0,3.0,2,2,...,5,4,3,4,4,4,5,0.000000,0.000000,1
3,0,0,2.0,0,0,0.0,3.0,8.0,5,5,...,2,2,5,3,1,4,2,2.484907,2.302585,0
4,1,0,6.0,0,0,0.0,5.0,5.0,3,3,...,3,3,4,4,3,3,3,0.000000,0.000000,1


In [91]:
data['Class'].unique()
data.nunique()

Gender                                 2
Customer Type                          2
Age                                    9
Type of Travel                         2
Class                                  3
Flight Distance                        9
Inflight wifi service                  6
Departure/Arrival time convenient      6
Ease of Online booking                 6
Gate location                          6
Food and drink                         6
Online boarding                        6
Seat comfort                           6
Inflight entertainment                 6
On-board service                       6
Leg room service                       6
Baggage handling                       5
Checkin service                        6
Inflight service                       6
Cleanliness                            6
Departure Delay in Minutes           466
Arrival Delay in Minutes             473
satisfaction                           2
dtype: int64

In [92]:
# chuẩn hóa dữ liệu Feature Scaling
scaler = StandardScaler()
data.iloc[:, :-1] = scaler.fit_transform(data.iloc[:, :-1])
# iloc số dòng, số cột

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_22016\2776714092.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 1.01487815  1.01487815 -0.98533997 ... -0.98533997  1.01487815
 -0.98533997]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.iloc[:, :-1] = scaler.fit_transform(data.iloc[:, :-1])
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_22016\2776714092.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.47342178  2.11228136 -0.47342178 ... -0.47342178 -0.47342178
 -0.47342178]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.iloc[:, :-1] = scaler.fit_transform(data.iloc[:, :-1])
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_22016\2776714092.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pa

In [93]:
data.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1.014878,-0.473422,-1.846098,1.493951,2.262940,-0.883671,0.257148,0.760442,0.173445,-1.546267,...,1.230787,0.479357,-0.266574,0.311762,0.547894,1.153946,1.304487,1.251360,1.028818,0
1,1.014878,2.112281,-0.714910,-0.669366,-0.955724,-0.883671,0.257148,-0.724123,0.173445,0.018048,...,-1.767616,-1.851475,1.252898,-0.535681,-1.821437,0.304086,-1.740402,-0.334224,0.416036,0
2,-0.985340,-0.473422,-0.714910,-0.669366,-0.955724,0.215023,-0.568380,-0.724123,-0.539957,-0.764109,...,1.230787,0.479357,-0.266574,0.311762,0.547894,0.304086,1.304487,-0.762709,-0.778138,1
3,-0.985340,-0.473422,-0.714910,-0.669366,-0.955724,-0.883671,-0.568380,1.131584,1.600249,1.582364,...,-1.018015,-1.074531,1.252898,-0.535681,-1.821437,0.304086,-0.979180,0.773394,0.634922,0
4,1.014878,-0.473422,1.547467,-0.669366,-0.955724,-0.883671,0.257148,0.018160,0.173445,0.018048,...,-0.268414,-0.297587,0.493162,0.311762,-0.241883,-0.545774,-0.217958,-0.762709,-0.778138,1


In [94]:
# chia dữ liệu train và test
train = data.drop('satisfaction', axis=1)
test = data['satisfaction']

In [95]:
# chọn hệ số tương quan
threadhold = 0.15
correlation = data.corr()['satisfaction']
selected_features = correlation[(correlation > threadhold) | (correlation < -threadhold)].index
selected_features = selected_features.drop('satisfaction')
train = train[selected_features]

In [96]:
# chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(train, test, test_size=0.3, random_state=42)

In [ ]:
# Huấn luyện mô hình SVM
svm_model = SVC(kernel='poly', random_state=42, probability=True)
svm_model.fit(X_train, y_train)

In [ ]:
# Dự đoán
y_train_pred = svm_model.predict(X_train)
y_test_pred = svm_model.predict(X_test)

In [ ]:
# Đánh giá
print("Validation Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Validation Report:\n", classification_report(y_train, y_train_pred))
print("Test Report:\n", classification_report(y_test, y_test_pred))

In [ ]:
# Matrix confusion (display with heatmap)
cm_c = confusion_matrix(y_test, y_test_pred)
plt.figure(figsize=(10,7))
sns.heatmap(cm_c, annot=True, fmt='d')
plt.show()

In [ ]:
# Classification report
print(classification_report(y_test, y_test_pred))

In [ ]:
# Accuracy score
print('Accuracy: ', accuracy_score(y_test, y_test_pred))